In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# load all csv files
Raw_interactions = pd.read_csv('Data/RAW_interactions.csv')
Raw_recipes = pd.read_csv('Data/RAW_recipes.csv')
PP_recipes = pd.read_csv('Data/PP_recipes.csv')
PP_users = pd.read_csv('Data/PP_users.csv')
ingr_map = pd.read_pickle('Data/ingr_map.pkl')
ingr_map.head()

,raw_ingr,raw_words,processed,len_proc,replaced,count,id
0,"medium heads bibb or red leaf lettuce, washed,...",13,"medium heads bibb or red leaf lettuce, washed,...",73,lettuce,4507,4308
1,mixed baby lettuces and spring greens,6,mixed baby lettuces and spring green,36,lettuce,4507,4308
2,romaine lettuce leaf,3,romaine lettuce leaf,20,lettuce,4507,4308
3,iceberg lettuce leaf,3,iceberg lettuce leaf,20,lettuce,4507,4308
4,red romaine lettuce,3,red romaine lettuce,19,lettuce,4507,4308


In [4]:
print('Raw_interactions Count: ', Raw_interactions.shape[0])
Raw_interactions.head()


Raw_interactions Count:  1132367


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [5]:
print('Raw_recipes Count: ', Raw_recipes.shape[0])
Raw_recipes.head(2)

Raw_recipes Count:  231637


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6


In [6]:
print('PP_recipes Count: ', PP_recipes.shape[0])
PP_recipes.head(2)

PP_recipes Count:  178265


,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."


In [7]:
print('PP_users Count: ', PP_users.shape[0])
PP_users.head(2)

PP_users Count:  25076


,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39


In [30]:
Words_in_review = {}
punc = ['.', ',', '!', '?', ';', ':', '(', ')', '[', ']', '{', '}', '/', '\\', '|', '-', '_', '+', '=', '*', '&', '^', '%', '$', '#', '@', '~', '`', '<', '>']


for rating, review in zip(Raw_interactions['rating'], Raw_interactions['review']):
    if type(review) != str:
        continue
    for j in review.split():
        j = j.lower()
        # remove punctuation
        for k in punc:
            j = j.strip(k)
        if j == '':
            continue
        if j in Words_in_review:
            Words_in_review[j].append(rating)
        else:
            Words_in_review[j] = [rating]

print('Words_in_review Count: ', len(Words_in_review))

Words_in_review Count:  438086


In [33]:
word_rating = {}
for word in Words_in_review:
    word_rating[word] = np.mean(Words_in_review[word]) if len(Words_in_review[word]) > 10 else 0

In [36]:
# sort by rating
word_rating = {k: v for k, v in sorted(word_rating.items(), key=lambda item: item[1], reverse=True)}
print('Top 20 words with highest rating: ', list(word_rating.items())[0:20])

Top 20 words with highest rating:  [('it!...kitten:', 5.0), ('recipeâ€”a', 5.0), ('perfect...the', 5.0), ('deely', 5.0), ('firefighter', 5.0), ('mexico/tex', 5.0), ('cake!"', 5.0), ('triples', 5.0), ('cathi', 5.0), ('thoes', 5.0), ('recipe.<br/>**made', 5.0), ('"we\'re', 5.0), ('coasty', 5.0), ('re-rolled', 5.0), ('tasties', 5.0), ('44126', 5.0), ('ape', 5.0), ('marg!...kitten:', 5.0), ('yell', 5.0), ('tajine', 5.0)]
